# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 8:30AM,257090,10,0086368743,ISDIN Corporation,Released
1,Feb 10 2023 8:30AM,257090,10,0086368746,ISDIN Corporation,Released
2,Feb 10 2023 8:30AM,257090,10,0086368748,ISDIN Corporation,Released
3,Feb 10 2023 8:30AM,257090,10,0086368747,ISDIN Corporation,Released
4,Feb 10 2023 8:30AM,257090,10,0086368749,ISDIN Corporation,Released
5,Feb 10 2023 8:30AM,257090,10,0086368745,ISDIN Corporation,Released
6,Feb 10 2023 8:30AM,257090,10,0086368744,ISDIN Corporation,Released
7,Feb 10 2023 8:30AM,257091,10,0086369222,ISDIN Corporation,Executing
8,Feb 10 2023 8:30AM,257091,10,0086369220,ISDIN Corporation,Released
9,Feb 10 2023 8:30AM,257091,10,0086369223,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,257094,Released,1
39,257095,Released,1
40,257096,Released,1
41,257097,Released,1
42,257098,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257094,NaN,NaN,1.0
257095,NaN,NaN,1.0
257096,NaN,NaN,1.0
257097,NaN,NaN,1.0
257098,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256934,22.0,18.0,3.0
256950,0.0,0.0,2.0
256993,1.0,0.0,2.0
257004,0.0,0.0,1.0
257010,2.0,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256934,22,18,3
256950,0,0,2
256993,1,0,2
257004,0,0,1
257010,2,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256934,22,18,3
1,256950,0,0,2
2,256993,1,0,2
3,257004,0,0,1
4,257010,2,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256934,22,18,3
1,256950,,,2
2,256993,1,,2
3,257004,,,1
4,257010,2,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation
7,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation
15,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation
23,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation
24,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation
29,Feb 10 2023 8:30AM,257096,10,ISDIN Corporation
49,Feb 10 2023 8:30AM,257088,10,"Citieffe, Inc."
58,Feb 10 2023 8:30AM,257087,10,Bio-PRF
65,Feb 10 2023 8:30AM,257080,10,Bio-PRF
71,Feb 10 2023 8:30AM,257094,10,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation,,,31
1,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation,,4,9
2,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,,,4
3,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,,,8
4,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,,,1
5,Feb 10 2023 8:30AM,257096,10,ISDIN Corporation,,,1
6,Feb 10 2023 8:30AM,257088,10,"Citieffe, Inc.",,,1
7,Feb 10 2023 8:30AM,257087,10,Bio-PRF,,,7
8,Feb 10 2023 8:30AM,257080,10,Bio-PRF,,,5
9,Feb 10 2023 8:30AM,257094,10,SugarBear,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation,31,,
1,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation,9,4,
2,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4,,
3,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8,,
4,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1,,
5,Feb 10 2023 8:30AM,257096,10,ISDIN Corporation,1,,
6,Feb 10 2023 8:30AM,257088,10,"Citieffe, Inc.",1,,
7,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7,,
8,Feb 10 2023 8:30AM,257080,10,Bio-PRF,5,,
9,Feb 10 2023 8:30AM,257094,10,SugarBear,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation,31,,
1,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation,9,4,
2,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4,,
3,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8,,
4,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation,31.0,NaN,NaN
1,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation,9.0,4.0,NaN
2,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4.0,NaN,NaN
3,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8.0,NaN,NaN
4,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 8:30AM,257090,10,ISDIN Corporation,31.0,0.0,0.0
1,Feb 10 2023 8:30AM,257091,10,ISDIN Corporation,9.0,4.0,0.0
2,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4.0,0.0,0.0
3,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8.0,0.0,0.0
4,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5655394,140.0,4.0,3.0
12,514137,2.0,0.0,0.0
15,1027917,13.0,18.0,22.0
19,1542422,35.0,3.0,0.0
20,257027,1.0,0.0,0.0
22,514075,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5655394,140.0,4.0,3.0
1,12,514137,2.0,0.0,0.0
2,15,1027917,13.0,18.0,22.0
3,19,1542422,35.0,3.0,0.0
4,20,257027,1.0,0.0,0.0
5,22,514075,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,140.0,4.0,3.0
1,12,2.0,0.0,0.0
2,15,13.0,18.0,22.0
3,19,35.0,3.0,0.0
4,20,1.0,0.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,140.0
1,12,Released,2.0
2,15,Released,13.0
3,19,Released,35.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,22
Status,,,,,,
Completed,3.0,0.0,22.0,0.0,0.0,0.0
Executing,4.0,0.0,18.0,3.0,0.0,0.0
Released,140.0,2.0,13.0,35.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,22
0,Completed,3.0,0.0,22.0,0.0,0.0,0.0
1,Executing,4.0,0.0,18.0,3.0,0.0,0.0
2,Released,140.0,2.0,13.0,35.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,22
0,Completed,3.0,0.0,22.0,0.0,0.0,0.0
1,Executing,4.0,0.0,18.0,3.0,0.0,0.0
2,Released,140.0,2.0,13.0,35.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()